# Portfolio Analysis

In [ ]:
!pip install xlrd
import datetime
import yfinance as yf
import pandas as pd

ticker_list = ['MSFT', 'SHOP.TO', '7203.T', 'HSBA.L', 'SGML.V']
tickers = yf.Tickers(' '.join(ticker_list))

field_list = ['shortName', 'currency', 'marketCap', 'impliedSharesOutstanding']

stat = pd.DataFrame([{f: tickers.tickers[t].info[f] for f in field_list} for t in ticker_list], index=ticker_list)

In [ ]:
port = pd.read_csv('sample.csv', index_col=0)
port

In [ ]:
last = pd.Series({t: tickers.tickers[t].history()['Close'].iloc[-1] for t in ticker_list})
stat['last'] = last
stat['marketCapLocal'] = stat['impliedSharesOutstanding'] * stat['last']
stat

# FIXME: Note GBp not GBP

In [ ]:
stat.info()

In [ ]:
date = datetime.datetime(year=2023, month=12, day=31)
base_currency = 'CAD'

fx_list = set(stat['currency']) - set([base_currency])
tickers = list(map(lambda fc: f'{base_currency}{fc}=X', fx_list))
fx = yf.Tickers(' '.join(tickers))
fx.tickers[tickers[0]].history()

In [ ]:
t = yf.Ticker('^GSPC')
px = t.history(period='max')['Close']
px.groupby(pd.Grouper(freq='ME')).last().pct_change().dropna()

In [ ]:
def adtv(ticker, end=datetime.datetime.today(), days=30):
    return yf.Ticker(ticker).history(period='max')['Volume'].loc[:end.strftime('%Y-%m-%d')].iloc[-days:].mean()

adtv('MSFT', days=10)
adtv('MSFT', end=datetime.datetime(year=2022, month=12, day=31))            # Expect 26906470.0
adtv('MSFT', end=datetime.datetime(year=2022, month=12, day=31), days=10)   # Expect 29035340.0

# Inclusive of both start/end date; Must provide both

# MSCI

MSCI Index Codes and corresponding Bloomberg Tickers, no factor indices

https://www.msci.com/www/product-documentation/bloomberg-ticker-list/0163827567

Factor performance in different time horizon, by region

https://www.msci.com/factor-index-scorecard/

In [ ]:
def get_last_business_day(d = datetime.datetime.today()):
    return (d - datetime.timedelta(days=max(0, d.weekday() - 4))).date()

# ACWI = 892400
codes = ['892400', '892400']
# Price = 'STRD', Net = 'NETR', Gross = 'GRTR'
# Currency
# YYYYMMDD, Must be trading date or server error
def get_msci(
        codes,
        end_date = get_last_business_day().strftime('%Y%m%d'),
        fx='USD',
        variant='STRD'):
    url = f'https://app2-nv.msci.com/products/service/index/indexmaster/downloadLevelData?output=INDEX_LEVELS&currency_symbol={fx}&index_variant={variant}&start_date=19691231&end_date={end_date}&data_frequency=END_OF_MONTH&baseValue=false&index_codes={",".join(codes)}'
    return pd.read_excel(url, skiprows=6, skipfooter=19).set_index('Date').squeeze().pct_change().dropna()

get_msci(codes)

## Liquidity